In [1]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

In [13]:
from carl.data import Ricker
r0 = Ricker(3.8)  # theta0
r1 = Ricker(5.0)  # theta1

In [14]:
X0 = np.array([r0.rvs(50) for i in range(10000)])
X1 = np.array([r1.rvs(50) for i in range(10000)])
X = np.vstack((X0, X1))
y = np.zeros(len(X))
y[len(X0):] = 1

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1000)
X_train, X_calibrate, y_train, y_calibrate = train_test_split(X_train, y_train, test_size=10000)

In [16]:
from keras.models import Sequential
from keras.layers import GRU, LSTM, Dense
from keras.optimizers import SGD
from keras.wrappers.scikit_learn import KerasClassifier

def make_model():
    model = Sequential()
    model.add(GRU(10, input_shape=(50, 1)))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    sgd = SGD(lr=0.01, clipnorm=40.)
    model.compile(loss="binary_crossentropy", optimizer=sgd)
    return model

clf = KerasClassifier(make_model, nb_epoch=50, batch_size=64, verbose=2)
clf.fit(X_train, y_train, validation_data=(X_test, y_test))

Train on 9000 samples, validate on 1000 samples
Epoch 1/50
1s - loss: 0.6749 - val_loss: 0.6477
Epoch 2/50
1s - loss: 0.6448 - val_loss: 0.6219
Epoch 3/50
1s - loss: 0.6238 - val_loss: 0.5981
Epoch 4/50
1s - loss: 0.5993 - val_loss: 0.5681
Epoch 5/50
1s - loss: 0.5652 - val_loss: 0.5218
Epoch 6/50
1s - loss: 0.4983 - val_loss: 0.4304
Epoch 7/50
1s - loss: 0.3644 - val_loss: 0.2564
Epoch 8/50
1s - loss: 0.1827 - val_loss: 0.1174
Epoch 9/50
1s - loss: 0.0952 - val_loss: 0.0710
Epoch 10/50
1s - loss: 0.0617 - val_loss: 0.0474
Epoch 11/50
1s - loss: 0.0917 - val_loss: 0.0748
Epoch 12/50
1s - loss: 0.0586 - val_loss: 0.0656
Epoch 13/50
1s - loss: 0.0502 - val_loss: 0.0551
Epoch 14/50
1s - loss: 0.0436 - val_loss: 0.0456
Epoch 15/50
1s - loss: 0.0360 - val_loss: 0.0423
Epoch 16/50
1s - loss: 0.0343 - val_loss: 0.0438
Epoch 17/50
1s - loss: 0.0279 - val_loss: 0.0357
Epoch 18/50
1s - loss: 0.0263 - val_loss: 0.0346
Epoch 19/50
1s - loss: 0.0234 - val_loss: 0.0264
Epoch 20/50
1s - loss: 0.0178 

In [39]:
from carl.ratios import ClassifierRatio
from carl.learning import CalibratedClassifierCV
from sklearn.model_selection import StratifiedShuffleSplit

ratio = ClassifierRatio(
    base_estimator=CalibratedClassifierCV(clf, cv="prefit", bins=5),
    random_state=0)
ratio.fit(X_calibrate, y_calibrate)

ClassifierRatio(base_estimator=CalibratedClassifierCV(base_estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x7f9955bffb00>,
            bins=5, cv='prefit', method='histogram'),
        random_state=0)

In [43]:
# Simple hypothesis test between log_r=3.8 (theta0) and log_r=5.0 (theta1)
for log_r in np.arange(3.8, 5+0.001, 0.05):
    r = Ricker(log_r)
    X_observed = np.array([r.rvs(50) for i in range(500)])
    print(log_r, ratio.nllr(X_observed))

3.8 -3285.28755746
3.85 -3277.58877072
3.9 -3251.31566865
3.95 -3131.86736167
4.0 -3084.24429593
4.05 -3016.08074289
4.1 -2691.86158704
4.15 -2472.31549715
4.2 -1846.86690266
4.25 -1001.07031248
4.3 -436.864195686
4.35 836.904756132
4.4 1161.79278761
4.45 2363.72819427
4.5 3385.28482551
4.55 4064.16986008
4.6 4699.91446769
4.65 5009.09270215
4.7 5279.46065842
4.75 5570.0090362
4.8 5708.59431486
4.85 5855.85451988
4.9 5919.23683583
4.95 6017.55468131
5.0 6022.58100206
